# Sistema de recomendación de películas - Content based filtering

El objetivo de este notebook es entender la similitud del coseno con un uso práctico. Tenemos los datos de usuarios, películas, ratings y tags.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#leer ficheros de ratings.csv y movies.csv, guardarlos en un Dataframe e inspeccionarlos

#implementar

In [5]:
ratings = pd.read_csv("data/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
movies = pd.read_csv("data/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [18]:
#Combinar el rating y películas para que podamos obtener los títulos de una forma más rápida
#Para ellos combinar las columnas del dataframe ratings (userId,movieId, rating) con las columnas de movies (movieId,title). Será necesarío hacer un leftjoin, donde la clave para el join será el atributo movieId
#guardar el resultado en la variable ratings

#implementar

Este código covierte los datos en una matriz donde las filas nos dan filas son Ids de usuario únicas y las columnas son Ids de películas únicas.
Por naturaleza, esta matriz sera sparse (mayoria de datos vacios), por lo que sustituiremos datos NAN por ceros.
pivot_table no es una función crítica, pero está bien saber que existe esta funcionalidad en pandas. Un blog ilustrativo: https://pbpython.com/pandas-pivot-table-explained.html

In [8]:
rp = ratings.pivot_table(columns = ['movieId'], index = ['userId'], values = 'rating')
rp = rp.fillna(0)
rp.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Convertimos el Dataframe a NumPy para una ejecución más rápida (no siempre es necesario). En este ejemplo utilizaremos bucles for por ser más ilustrativo, pero no deberíais iterar sobre arrays de Numpy, ya que está diseñado para hacer ejecuciones de una sola tirada.

Teniendo esta matriz como base, podemos calcular la **similaridad entre usuarios y películas**.

Recordemos cuál era la forma de calcular la distancia del coseno entre dos vectores:    

In [25]:
from scipy.spatial.distance import cosine
#El coseno del ángulo entre los dos vectores es 0.822.
a = np.asarray( [2, 1, 0, 2, 0, 1, 1, 1])
b = np.asarray( [2, 1, 1, 1, 1, 0, 1, 1])
print (1-cosine(a,b))

0.8215838362577491


Sigamos...



In [29]:
m, n = rp.shape
print(m)
rp_mat = rp.as_matrix()

# Tenemos que generar una matriz donde se almacene la similaridad entre usuarios

users_mat = np.zeros((m, m))

#En cada fila de user_mat almacenaremos la similaridad de un usuario respecto a los demas usuarios

#implementar

pd_users = pd.DataFrame(users_mat,index =rp.index ,columns= rp.index)

print(pd_users)

610


C:\Users\aaguirre\.conda\envs\AnalisisExploratorio\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


userId       1         2         3         4         5         6         7    \
userId                                                                         
1       0.000000  0.027283  0.059720  0.194395  0.129080  0.128152  0.158744   
2       0.027283  0.000000  0.000000  0.003726  0.016614  0.025333  0.027585   
3       0.059720  0.000000  0.000000  0.002251  0.005020  0.003936  0.000000   
4       0.194395  0.003726  0.002251  0.000000  0.128659  0.088491  0.115120   
5       0.129080  0.016614  0.005020  0.128659  0.000000  0.300349  0.108342   
...          ...       ...       ...       ...       ...       ...       ...   
606     0.164191  0.028429  0.012993  0.200395  0.106435  0.102123  0.200035   
607     0.269389  0.012948  0.019247  0.131746  0.152866  0.162182  0.186114   
608     0.291097  0.046211  0.021128  0.149858  0.135535  0.178809  0.323541   
609     0.093572  0.027565  0.000000  0.032198  0.261232  0.214234  0.090840   
610     0.145321  0.102427  0.032119  0.

A continuación, implementaremos una función que nos devuelva los N usuarios más parecidos de un usuario, pasándole la Id del usuario y el número de usuarios parecidos a devolver como parámetros. Para desarrollar las siguientes dos funciones, podéis utilizar las funciones loc e iloc de pandas, las cuales nos permiten indexar DataFrames: 

- Doc. oficial: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html
- ejemplos: https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/

In [30]:
def topn_simusers(uid,n):
    
    #implementar
    return pd.DataFrame(topn_users)

print (topn_simusers(uid=17,n=10))

Usuarios similares al usuario: 17
           score
userId          
16      0.456096
400     0.452319
434     0.452304
247     0.438913
399     0.414196
362     0.405180
549     0.404689
131     0.403409
72      0.392264
464     0.386456


La tarea no acaba con encontrar usuarios similares. El objetivo sería ver cuáles son las películas mejor valoradas por cada usuario. Desarrollemos una función que devuelva las películas mejor valoradas por cada usuario:

In [31]:
def topn_movieratings(uid = 355,n_ratings=10):
    uid_ratings = ratings.loc[ratings['userId']==uid]
    uid_ratings = uid_ratings.sort_values(by='rating',ascending = False)
    print ("Top",n_ratings ,"de valoraciones de películas del usuario:",uid)
    return uid_ratings.iloc[:n_ratings,]
print (topn_movieratings(uid=596,n_ratings=10))

Top 10 movie ratings of user: 596
       userId  movieId  rating   timestamp  \
91864     596     3000     5.0  1535708657   
91974     596    33649     5.0  1535829905   
92107     596   122906     5.0  1535708624   
91905     596     4878     5.0  1535708338   
92109     596   122916     5.0  1535627433   
91928     596     5971     5.0  1535708665   
92084     596   110102     5.0  1535709675   
91837     596     2288     5.0  1535827560   
91969     596    31658     5.0  1535708584   
92100     596   122882     5.0  1535708481   

                                                   title  
91864           Princess Mononoke (Mononoke-hime) (1997)  
91974                                 Saving Face (2004)  
92107                               Black Panther (2017)  
91905                                Donnie Darko (2001)  
92109                              Thor: Ragnarok (2017)  
91928       My Neighbor Totoro (Tonari no Totoro) (1988)  
92084         Captain America: The Winter Sold

Basándonos en este enfoque, podríamos seguir varios caminos:
a) podemos sugerir películas de los x usuarios que más se parezcan a un usuario, eliminando aquellas que ya ha visto el usuario
b) una forma más avanzada para una recomendación sería obtener una matriz de similaridad entre películas (proceso similar al que hemos seguido con los usuarios), y recomendar aquellas películas parécidas a las que nos gusten.

Implementad la opción que más os interese.